In [ ]:

minimal.parser.add_argument("--fir_length", type=int, default=8, 
                           help="Number of chunks in FIR filter")
minimal.parser.add_argument("--mu", type=float, default=0.01, 
                           help="LMS learning rate (0.001-0.1)")

Las variables para poder adaptar el código a nuestro entorno serán:
- mu: modifica la velocidad a la que aprende el código a adaptarse (si es muy alto podría ser inestable y si es muy bajo muy lento)
- fir_length: longitud de eco atrás que tendremos en cuenta (numero de chunks que vamos a usar)

In [ ]:
def __init__(self):
        super().__init__()
        logging.info(__doc__)
        
        self.played_chunk_history = []
    
        self.fir_length = minimal.args.fir_length
        self.fir_coeffs = np.zeros(self.fir_length, dtype=np.float32)
        self.mu = minimal.args.mu
       

Definimos la variables y los valores por defecto que vamos a usar:
- fir_length que es la longitud de chunks que vamos a tener en cuenta 
- los coeficientes iniciales en 0, un array del tamaño de los chunks que vamos a tener en cuenta pero en 0
- mu la variable que nos ayudará a cambiar la velocidad de aprendizaje


In [ ]:
 chunk = self.unbuffer_next_chunk()
        if chunk is None:
            chunk = self.zero_chunk

        # Guardar historial
        self.played_chunk_history.append(chunk.copy()) 
        if len(self.played_chunk_history) > self.fir_length:
            self.played_chunk_history.pop(0)


Escogemos el primer chunk y si es nulo le damos valor 0. Copiamos todos los chunks que tengamos anteriormente en el historial y si el tamaño es mayor de lo que vamos a considerar eliminamos los más antiguos.
Este apartado enseña como se añade y se eliminan los chunks del historial para no hacer uso de los más antiguos y tener en cuenta los nuevos.

In [ ]:

 # Convertir a float32 en [-1,1]
        ADC_float = ADC.astype(np.float32) / 32768.0 #señal de micrófono actual
        played_chunks_float = [c.astype(np.float32) / 32768.0 for c in self.played_chunk_history]
    

ADC_float es la señal de micrófono actual convertida a float32.
played_chuncks_float es el historial de chunks anteriores, pero convertidos a float para poder hacer operaciones y finalmente que puedan ser reproducidos.
Es importante recalcar que el propio historial de chunks anterior no es reescrito, sino que played_chunks_float es un nuevo array de floats

In [ ]:
echo_estimation = np.zeros_like(ADC_float)#crea array de 0 del tamaño de ADC
        for k in range(len(played_chunks_float)):
            echo_estimation += self.fir_coeffs[k] * played_chunks_float[-(k+1)]

En esta instrucción estimamos el eco que guardaremos en la variable "echo_estimation"
La comenzamos inicializando con el tamaño de ADC, la señal actual, pero llena de 0.
A continuación la vamos completando con la multiplicación del coeficiente (de cada uno de los chunks) por cada uno de los chunks.

In [ ]:
        error = ADC_float - echo_estimation #a la señal actual le resta la estimación del eco 

        # LMS por chunk promedio (evita ValueError) calcula el coeficiente FIR para cada chunk para que aprenda automáticamente
        for k in range(len(played_chunks_float)):
            x = played_chunks_float[-(k+1)] #para ir de más reciente a menos 
            self.fir_coeffs[k] += self.mu * np.mean(error * x) / (np.mean(x**2)+1e-6)
      

error es la señal limpia que vamos a reproducir, por ello es la resta de la señal actual menos la estimación del eco.
El algoritmo LMS (Least Mean Squares) se encarga de actualizar los coeficientes FIR para cada chunk, de manera que el sistema pueda adaptarse y aprender a eliminar el eco de forma automática

In [ ]:
 filtered_ADC = np.clip(error * 32768.0, -32768, 32767).astype(np.int16)

        # Atenuar chunk reproducido para reducir feedback directo
        chunk_to_play = (chunk.astype(np.float32) * 0.2).astype(np.int16)
        self.play_chunk(DAC, chunk_to_play)

        # Enviar señal limpia
        packed_chunk = self.pack(self.chunk_number, filtered_ADC)
        self.send(packed_chunk)
    

Por último, volvemos a normalizar la señal que vamos a reproducir sin eco.
Usamos el chunk y lo atenuamos multiplicándolo por 0.2 de esta manera evitaríamos que cree más feedback.
Empaquetamos el chunk con la señal limpia y lo enviamos para que lo reproduzca.